In [102]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import time
import numpy as np

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [103]:
#Using Pandas to create dataframe from an excel file
df = pd.read_excel("Compiled_Nashville_September_Property_Sales_GetCoordsFinal.xlsx")
df['Sale Price'] = df['Sale Price'].apply(lambda x: "${:.1f}k".format((x/1000)))
df.head()

,Parcel ID,Land Use,Property Address,Property City,Sale Date,Sale Price,Legal Reference,Sold As Vacant,Multiple Parcels Involved in Sale,Zone,Coordinates
0,053 05 0A 008.00,SINGLE FAMILY,1100 CHEYENNE CIR,MADISON,2021-09-07,$304.0k,20210921-0126315,No,No,6,"36.2524142, -86.6706970"
1,006 00 0 010.00,SINGLE FAMILY,3450 GREER RD,GOODLETTSVILLE,2021-09-17,$405.0k,20210920-0125863,No,No,5,"36.3835361, -86.7832459"
2,004 00 0 075.00,SINGLE FAMILY,8565 WHITES CREEK PIKE,JOELTON,2021-09-22,$666.5k,20211001-0131984,No,No,5,"36.3811685, -86.8945705"
3,004 00 0 193.00,SINGLE FAMILY,8560 COOPERTOWN RD,JOELTON,2021-09-20,$620.0k,20210924-0128377,No,No,5,"36.3808100, -86.9061216"
4,004 00 0 139.00,SINGLE FAMILY,8409 RILEY ADCOCK RD,JOELTON,2021-09-17,$227.5k,20210923-0127923,No,No,5,"36.3729140, -86.8836440"


In [104]:
#Getting the Latitude and Longitude from the Coordinates Column
df[["Latitude","Longitude"]] = df["Coordinates"].str.split(",", expand = True)
df.head()

,Parcel ID,Land Use,Property Address,Property City,Sale Date,Sale Price,Legal Reference,Sold As Vacant,Multiple Parcels Involved in Sale,Zone,Coordinates,Latitude,Longitude
0,053 05 0A 008.00,SINGLE FAMILY,1100 CHEYENNE CIR,MADISON,2021-09-07,$304.0k,20210921-0126315,No,No,6,"36.2524142, -86.6706970",36.2524142,-86.6706970
1,006 00 0 010.00,SINGLE FAMILY,3450 GREER RD,GOODLETTSVILLE,2021-09-17,$405.0k,20210920-0125863,No,No,5,"36.3835361, -86.7832459",36.3835361,-86.7832459
2,004 00 0 075.00,SINGLE FAMILY,8565 WHITES CREEK PIKE,JOELTON,2021-09-22,$666.5k,20211001-0131984,No,No,5,"36.3811685, -86.8945705",36.3811685,-86.8945705
3,004 00 0 193.00,SINGLE FAMILY,8560 COOPERTOWN RD,JOELTON,2021-09-20,$620.0k,20210924-0128377,No,No,5,"36.3808100, -86.9061216",36.3808100,-86.9061216
4,004 00 0 139.00,SINGLE FAMILY,8409 RILEY ADCOCK RD,JOELTON,2021-09-17,$227.5k,20210923-0127923,No,No,5,"36.3729140, -86.8836440",36.3729140,-86.8836440


In [105]:
#Converting Lat and Long to a float
df['Latitude'] = pd.to_numeric(df['Latitude'],errors='coerce')
df['Longitude'] = pd.to_numeric(df['Longitude'],errors='coerce')
df.dtypes

Parcel ID                                    object
Land Use                                     object
Property Address                             object
Property City                                object
Sale Date                            datetime64[ns]
Sale Price                                   object
Legal Reference                              object
Sold As Vacant                               object
Multiple Parcels Involved in Sale            object
Zone                                          int64
Coordinates                                  object
Latitude                                    float64
Longitude                                   float64
dtype: object

In [107]:
# Create a new Data Frame that contains information that is needed.
places_df = df[["Property Address", "Property City", "Sale Price", "Latitude", "Longitude"]].copy()
places_df["Restaurants"] = ""
places_df["Supermarkets"] = ""
places_df["Liquor Store"] = ""
places_df["Schools"] = ""
places_df["Churches"] = ""
places_df.head(10)
#Adding the following: 
# restaurant 
# supermarket 
# liquor_store 
# school 
# church ------- Alecia's List
#-------------
# convenience_store 
# drugstore 
# fire_station
# hospital 
# bar
#This is for Chris

,Property Address,Property City,Sale Price,Latitude,Longitude,Restaurants,Supermarkets,Liquor Store,Schools,Churches
0,1100 CHEYENNE CIR,MADISON,$304.0k,36.252414,-86.670697,,,,,
1,3450 GREER RD,GOODLETTSVILLE,$405.0k,36.383536,-86.783246,,,,,
2,8565 WHITES CREEK PIKE,JOELTON,$666.5k,36.381169,-86.894571,,,,,
3,8560 COOPERTOWN RD,JOELTON,$620.0k,36.380810,-86.906122,,,,,
4,8409 RILEY ADCOCK RD,JOELTON,$227.5k,36.372914,-86.883644,,,,,
5,7840 WHITES CREEK PIKE,JOELTON,$575.0k,36.355360,-86.878055,,,,,
6,3560 BAXTER RD,JOELTON,$805.0k,36.352787,-86.885008,,,,,
7,7564 BIDWELL RD,JOELTON,$313.5k,36.343657,-86.893232,,,,,
8,208 A SWIFT DR,GOODLETTSVILLE,$333.0k,36.335490,-86.720321,,,,,
9,109 SWIFT DR,GOODLETTSVILLE,$350.0k,36.334522,-86.719428,,,,,


In [117]:
#making a dummy table
five_df = places_df.head(5)

#Creating a Function
def get_num_businesses(columnName,placetype, table):
    #Setting up the gmaps search and nearby proximity
    params  = {
        "radius": 5000,
        "type": placetype,
        "key": g_key
        }


    for index, row in table.iterrows():
    #Getting Latitude and Longitude
        lat = row["Latitude"]
        lng = row["Longitude"]
        params["location"] = f"{lat},{lng}"

    #Getting base url
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        places = requests.get(base_url, params=params).json()

        time.sleep(1)

        try:        
            table.loc[index, columnName] = len(places["results"])
        except:
            print("No place found... skipping")



# for index, row in fast_food_df.iterrows():
# #Getting Latitude and Longitude
#     lat = row["Latitude"]
#     lng = row["Longitude"]
#     params["location"] = f"{lat},{lng}"

# #Getting base url
#     base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#     time.sleep(3)

#     food_places = requests.get(base_url, params=params).json()

#     try:
#         fast_food_df.loc[index, "Fast Food"] = food_places["results"][0]["name"]
#     except (IndexError):
#         print("No fast food found... skipping")


In [109]:
#Our Dummy Table
five_df

,Property Address,Property City,Sale Price,Latitude,Longitude,Restaurants,Supermarkets,Liquor Store,Schools,Churches
0,1100 CHEYENNE CIR,MADISON,$304.0k,36.252414,-86.670697,,,,,
1,3450 GREER RD,GOODLETTSVILLE,$405.0k,36.383536,-86.783246,,,,,
2,8565 WHITES CREEK PIKE,JOELTON,$666.5k,36.381169,-86.894571,,,,,
3,8560 COOPERTOWN RD,JOELTON,$620.0k,36.380810,-86.906122,,,,,
4,8409 RILEY ADCOCK RD,JOELTON,$227.5k,36.372914,-86.883644,,,,,


In [112]:
get_num_businesses("Restaurants","restaurant",places_df)

In [118]:
get_num_businesses("Supermarkets","supermarket",places_df)

In [120]:
get_num_businesses("Liquor Store","liquor_store",places_df)

In [122]:
get_num_businesses("Schools","school",places_df)

In [123]:
get_num_businesses("Churches","church",places_df)

In [98]:
#Calling the function
get_num_businesses("Restaurants","restaurant",five_df)
get_num_businesses("Supermarkets","supermarket",five_df)
get_num_businesses("Liquor Store","liquor_store",five_df)
get_num_businesses("Schools","school",five_df)
get_num_businesses("Churches","church",five_df)


C:\Users\Alecia\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [124]:
#Testing Output
places_df

,Property Address,Property City,Sale Price,Latitude,Longitude,Restaurants,Supermarkets,Liquor Store,Schools,Churches
0,1100 CHEYENNE CIR,MADISON,$304.0k,36.252414,-86.670697,20,7,8,20,20
1,3450 GREER RD,GOODLETTSVILLE,$405.0k,36.383536,-86.783246,16,0,2,9,11
2,8565 WHITES CREEK PIKE,JOELTON,$666.5k,36.381169,-86.894571,1,0,1,1,10
3,8560 COOPERTOWN RD,JOELTON,$620.0k,36.380810,-86.906122,3,0,2,1,10
4,8409 RILEY ADCOCK RD,JOELTON,$227.5k,36.372914,-86.883644,1,0,0,1,5
...,...,...,...,...,...,...,...,...,...,...
1458,4232 DYSANT ALY,NOLENSVILLE,$460.0k,35.985114,-86.662269,20,2,5,20,20
1459,1983 CAROTHERS RD,NOLENSVILLE,$497.3k,35.984074,-86.622866,20,1,3,10,20
1460,307 MOIRA CIR,NOLENSVILLE,$300.4k,35.983315,-86.622208,20,1,3,10,20
1461,4154 ALVA LN,NOLENSVILLE,$500.0k,35.983120,-86.663883,20,2,5,20,20


In [134]:
#Dropping Columns from the table

places_df

,Property Address,Property City,Sale Price,Latitude,Longitude,Restaurants,Supermarkets,Liquor Store,Schools,Churches
0,1100 CHEYENNE CIR,MADISON,$304.0k,36.252414,-86.670697,20,7,8,20,20
1,3450 GREER RD,GOODLETTSVILLE,$405.0k,36.383536,-86.783246,16,0,2,9,11
2,8565 WHITES CREEK PIKE,JOELTON,$666.5k,36.381169,-86.894571,1,0,1,1,10
3,8560 COOPERTOWN RD,JOELTON,$620.0k,36.380810,-86.906122,3,0,2,1,10
4,8409 RILEY ADCOCK RD,JOELTON,$227.5k,36.372914,-86.883644,1,0,0,1,5
...,...,...,...,...,...,...,...,...,...,...
1458,4232 DYSANT ALY,NOLENSVILLE,$460.0k,35.985114,-86.662269,20,2,5,20,20
1459,1983 CAROTHERS RD,NOLENSVILLE,$497.3k,35.984074,-86.622866,20,1,3,10,20
1460,307 MOIRA CIR,NOLENSVILLE,$300.4k,35.983315,-86.622208,20,1,3,10,20
1461,4154 ALVA LN,NOLENSVILLE,$500.0k,35.983120,-86.663883,20,2,5,20,20


In [135]:
#Exporting the File to CSV
places_df.to_csv('places_first_five.csv')

In [99]:
#Testing the output
five_df

,Property Address,Property City,Sale Price,Latitude,Longitude,Restaurants,Supermarkets,Liquor Store,Schools,Churches
0,1100 CHEYENNE CIR,MADISON,$304.0k,36.252414,-86.670697,20,7,8,20,20
1,3450 GREER RD,GOODLETTSVILLE,$405.0k,36.383536,-86.783246,16,0,2,9,11
2,8565 WHITES CREEK PIKE,JOELTON,$666.5k,36.381169,-86.894571,1,0,1,1,10
3,8560 COOPERTOWN RD,JOELTON,$620.0k,36.380810,-86.906122,3,0,2,1,10
4,8409 RILEY ADCOCK RD,JOELTON,$227.5k,36.372914,-86.883644,1,0,0,1,5
